# Part 1
## Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

* Q1 ans:

SELECT name 
FROM Facilities
WHERE membercost>0;


## Q2: How many facilities do not charge a fee to members? 

* Q2 ans:

SELECT count(*) 
FROM Facilities
WHERE membercost=0;


## Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question. 

* ans:

SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost >0
	AND membercost < monthlymaintenance*0.2;



## Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator. */

* ans:

SELECT *
FROM Facilities
WHERE facid
IN ( 1, 5 );



## Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

* ans:

SELECT name, monthlymaintenance,
	CASE WHEN monthlymaintenance > 100 THEN 'expensive'
		ELSE 'cheap' END AS label
FROM Facilities;



##  Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

* ans:

SELECT firstname, surname
FROM Members
WHERE joindate=
	(SELECT MAX(joindate)
    FROM Members);



##  Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

* ans: 

SELECT mbf.member, mbf.name
FROM(
    SELECT DISTINCT memid, CONCAT(firstname, ' ', surname) AS member, f.name
	FROM Members AS m
	LEFT JOIN Bookings AS b
	USING(memid)
	LEFT JOIN Facilities AS f
	USING(facid)
	WHERE f.name LIKE '%Tennis Court%'
	ORDER BY member) AS mbf;



## Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. 
 * ans:
 
 SELECT f.name, 
	CASE WHEN b.memid != 0 THEN CONCAT(m.surname, ' ', m.firstname) 
		ELSE 'GUEST' END AS member_name, 
	CASE WHEN b.memid != 0 THEN f.membercost
		ELSE f.guestcost END AS cost
FROM Bookings AS b
    LEFT JOIN Facilities AS f
    USING(facid)
    LEFT JOIN Members as m
    USING(memid)
    WHERE (guestcost > 30 AND membercost > 30)
	AND (starttime >= '20120914 00:00:00' OR starttime < '20120915 00:00:00')
    ORDER BY COST DESC;



## Q9: This time, produce the same result as in Q8, but using a subquery. 

* ans:

SELECT fb.name,
   CASE WHEN m.memid = 0 THEN 'GUEST'
	   ELSE CONCAT(firstname,' ', surname) END AS member,
   CASE WHEN m.memid = 0 THEN fb.guestcost
	   ELSE fb.membercost END AS cost
FROM Members AS m
LEFT JOIN (
	SELECT name, membercost, guestcost, memid
	FROM Facilities AS f
	LEFT JOIN 
    	Bookings AS b
	USING(facid)
	WHERE (starttime >= '2012-09-14' AND starttime < '2012-09-15') 
		AND (guestcost>30 OR membercost>30)) AS fb
USING(memid)
ORDER BY cost DESC;

## Import sqiite3

In [2]:
import sqlite3

## Read-only connection

In [11]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db', uri=True)

## Create a cursor object

In [12]:
curs = conn.cursor()

## Use the cursor’s fetchall() method to get the tables listed in sqlite_master

In [15]:
curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('Bookings',), ('Facilities',), ('Members',)]

In [18]:
for row in curs.execute('SELECT * FROM Facilities'):
    print(row)

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [59]:
bookings = [row for row in curs.execute('SELECT * FROM Bookings')]
for i in range(10):
    print(bookings[i])

(0, 3, 1, '2012-07-03 11:00:00', 2)
(1, 4, 1, '2012-07-03 08:00:00', 2)
(2, 6, 0, '2012-07-03 18:00:00', 2)
(3, 7, 1, '2012-07-03 19:00:00', 2)
(4, 8, 1, '2012-07-03 10:00:00', 1)
(5, 8, 1, '2012-07-03 15:00:00', 1)
(6, 0, 2, '2012-07-04 09:00:00', 3)
(7, 0, 2, '2012-07-04 15:00:00', 3)
(8, 4, 3, '2012-07-04 13:30:00', 2)
(9, 4, 0, '2012-07-04 15:00:00', 2)


# Q10: Produce a list of facilities with a total revenue less than 10000.

In [58]:
query = 'SELECT name, SUM(CASE WHEN memid = 1 THEN membercost ELSE guestcost END) AS revenue FROM Bookings LEFT JOIN Facilities USING(facid) GROUP BY name HAVING revenue<10000 ORDER BY revenue DESC'
for row in curs.execute(query):
    print(row)

('Tennis Court 2', 9345)
('Massage Room 2', 8880)
('Squash Court', 7504.0)
('Pool Table', 4040)
('Badminton Court', 3890.5)
('Snooker Table', 2160)
('Table Tennis', 1875)


# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [ ]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 
query = "SELECT firstname || ' ' || surname AS member FROM Members WHERE memid != 0 ORDER BY member"
for row in curs.execute(query):
    print(row)

# Q12: Find the facilities with their usage by member, but not guests

In [75]:
query = 'SELECT name, firstname || " " || surname AS member FROM Facilities LEFT JOIN Bookings USING(facid) LEFT JOIN Members AS m USING(memid) WHERE memid != 0'
rows = [row for row in curs.execute(query)]
for i in range(10):
    print(rows[i])

('Table Tennis', 'Darren Smith')
('Massage Room 1', 'Darren Smith')
('Snooker Table', 'Darren Smith')
('Pool Table', 'Darren Smith')
('Pool Table', 'Darren Smith')
('Tennis Court 1', 'Tracy Smith')
('Tennis Court 1', 'Tracy Smith')
('Massage Room 1', 'Tim Rownam')
('Squash Court', 'Darren Smith')
('Snooker Table', 'Tracy Smith')


# Q13: Find the facilities usage by month, but not guests 

In [94]:
query = 'SELECT name, COUNT(name) AS count, STRFTIME("%m",starttime) AS month FROM Facilities LEFT JOIN Bookings USING(facid) GROUP BY month, name'

for row in curs.execute(query):
    print(row)

('Badminton Court', 56, '07')
('Massage Room 1', 123, '07')
('Massage Room 2', 12, '07')
('Pool Table', 110, '07')
('Snooker Table', 75, '07')
('Squash Court', 75, '07')
('Table Tennis', 51, '07')
('Tennis Court 1', 88, '07')
('Tennis Court 2', 68, '07')
('Badminton Court', 146, '08')
('Massage Room 1', 224, '08')
('Massage Room 2', 40, '08')
('Pool Table', 291, '08')
('Snooker Table', 159, '08')
('Squash Court', 170, '08')
('Table Tennis', 147, '08')
('Tennis Court 1', 146, '08')
('Tennis Court 2', 149, '08')
('Badminton Court', 181, '09')
('Massage Room 1', 282, '09')
('Massage Room 2', 59, '09')
('Pool Table', 435, '09')
('Snooker Table', 210, '09')
('Squash Court', 195, '09')
('Table Tennis', 205, '09')
('Tennis Court 1', 174, '09')
('Tennis Court 2', 172, '09')
